In [ ]:
import os

os.chdir("../../")
from langchain.agents import create_agent

from initialization import credential_init

credential_init()

In [ ]:
from textwrap import dedent

from langchain.tools import BaseTool
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

# 這個 prompt 會讓 agent 像顧問主管 / 智庫分析師，而不是直接下場寫報告的研究員。
system_prompt = dedent("""\
You are an expert analytical agent specializing in international relations, geopolitics, and global industrial analysis.

Your core responsibilities are:

1. Problem Framing
   - Precisely interpret the user's question.
   - Identify the relevant countries, regions, industries, and time horizons involved.
   - Clarify implicit assumptions and constraints.

2. Analytical Decomposition
   - Break complex international or industrial issues into structured sub-problems
     (e.g. political, economic, technological, supply-chain, and strategic dimensions).
   - Determine what type of analysis is required (scenario analysis, risk assessment, strategic comparison, policy impact, etc.).

3. Tool Orchestration
   - Delegate deep analysis and solution generation to the SolutionTool when appropriate.
   - Provide the tool with a clear, well-scoped analytical query, including context and objectives.
   - Do NOT generate full solutions yourself if the task has been delegated to the tool.

4. Synthesis & Validation
   - Integrate the tool’s output into a coherent, logically consistent final response.
   - Ensure conclusions are evidence-based, internally consistent, and aligned with real-world constraints.

5. Analytical Standards
   - Prioritize accuracy, clarity, and structured reasoning over verbosity.
   - Avoid speculation without stating assumptions.
   - Maintain a neutral, professional, expert tone.

You act as the strategic brain of the system: you think, plan, and coordinate analysis, while specialized tools perform deep solution work.
Response in traditional Chinese (繁體中文)
""")




這個 system prompt 要讓 tool model 變成：

>- 冷靜、嚴謹、不聊天、只輸出分析與解法的專家模型

In [ ]:
solution_tool_system_prompt = dedent("""\
You are a senior expert in international relations, geopolitics, and global industry analysis.

Your task is to provide rigorous, structured, and actionable analysis based on the query provided.

Guidelines:
1. Analytical Rigor
   - Base your reasoning on established economic, political, and industrial frameworks.
   - Explicitly state assumptions when data is uncertain or scenario-based.

2. Structured Output
   - Organize responses using clear sections and logical progression.
   - When appropriate, include:
     - Background context
     - Key drivers and constraints
     - Stakeholder or country-level analysis
     - Risks and uncertainties
     - Strategic options or solutions

3. Solution-Oriented Thinking
   - Go beyond description; propose realistic and implementable solutions or policy options.
   - Evaluate trade-offs between alternatives.

4. Neutrality & Precision
   - Avoid ideological bias.
   - Do not speculate without justification.
   - Use precise, professional language.

You do not manage tools or conversations.
You do not ask follow-up questions unless critical information is missing.
Your sole responsibility is high-quality analysis and solution generation.
Response in traditional Chinese (繁體中文)
""")

In [ ]:
from langchain_core.runnables import Runnable

class SolutionTool(BaseTool):
    name:str = "SolutionTool" 
    description:str = dedent("""\
    A specialized analytical tool designed to generate rigorous, evidence-based analysis and actionable solutions.

    This tool is responsible for:
    - Conducting in-depth geopolitical, international relations, and global industry analysis.
    - Evaluating country-level strategies, industrial competitiveness, supply chains, and policy impacts.
    - Producing structured outputs such as:
      - Root-cause analysis
      - Scenario comparisons
      - Risk and opportunity assessments
      - Strategic recommendations and solution options
    
    Input should clearly specify:
    - The problem or question to be analyzed
    - Relevant countries, regions, or industries
    - The decision-making or strategic objective (e.g. policy design, investment, risk mitigation)
    
    The tool focuses on analytical rigor and solution quality, not conversation management or task orchestration.
    """)
    
    def _run(self, query: str):

        messages = [
            SystemMessage(content=solution_tool_system_prompt),
            HumanMessage(content=query)
        ]
        
        return  solution_llm.invoke(messages)
    
    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")

tools = [SolutionTool()]

solution_llm = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                          model_name="gpt-4.1", temperature=0, name='solution_tool_model')

model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                   model_name="gpt-4o", temperature=0, name='agent_model'
                  )

agent = create_agent(
    model=model,
    tools=tools,
    system_prompt=system_prompt,
    name = "analysis_agent"
)

In [ ]:
content = dedent("""\

請根據以下台美關稅與投資談判內容，分析對台灣的產業、社會與國際外交影響，並提出可能的風險與機會。請使用繁體中文回答，條理清楚，分點呈現，並保持分析深度：

### 已知內容：
1. 美國同意將對台灣出口商品的對等關稅降至 15%，比原本暫時性實施的 20% 暫定稅率更低。
2. 此 15% 的稅率不會再疊加（即不疊加現有最惠國稅率等），等同於美國對日本、韓國等主要貿易夥伴的等級。
3. 台灣的半導體和相關高科技產品在美國《232條款》下獲得「最優惠待遇」（Most Favored Nation treatment）。
4. 台灣企業（尤其是半導體、ICT、人工智慧等產業）承諾將在美國進行大規模投資，官方報導與美方公告提及至少美金數千億美元的投資承諾，包括直接擴廠與信用保證措施。
5. 美國商務部長盧特尼克在公開訪談中提到，美國政府的一個長期目標是讓全球約 40% 的台灣半導體供應鏈產能轉移到美國，以提升美國在先進晶片製造的自給自足程度並強化國家安全。
   - 在台美目前宣布的貿易協議與投資備忘錄（MOU）中，沒有具體規定企業必須把 40% 半導體產能搬到美國。協議主要內容是：台灣科技企業承諾將對美投資巨額資金（直接投資與信用保證合計約 5,000 億美元），作為交換美方將對台關稅調降並在特定情況下豁免《232條款》課徵的高額關稅。換句話說，40% 產能移轉是一個美方政策目標與談判籌碼，但目前沒有法律或協議文本要求台灣企業達成該具體比例。

### 分析要求：
1. **產業面**：分析對台灣半導體、高科技、ICT、AI等產業的影響，包括成本、供應鏈、競爭力與產業結構調整的風險與機會。
2. **社會面**：分析對台灣就業、市場薪資、社會輿論與政治角力的可能影響。
3. **國際外交面**：分析台美協議對台灣地緣政治位置、與其他主要貿易夥伴關係及區域產業競爭格局的影響。
4. 提出明確的 **風險與機會**。
5. 條理清楚、分點呈現，保持分析深度。


""")

# output = agent.invoke({"messages": HumanMessage(content=content)})

In [ ]:
output

In [ ]:
print(output["messages"][-1].content)

## 拆解問題 → 逐項回答 → 整合

In [ ]:
system_prompt = dedent("""\
You are an analytical orchestration agent.

For every complex question, you must:

1. 問題拆解
   - 將使用者問題拆解為明確、互不重疊的子問題。
   - 每個子問題需對應一個分析面向（政治、經濟、產業、風險等）。

2. 子問題處理
   - 對於需要專業分析的子問題，調用 SolutionTool。
   - 每次工具調用只處理一個子問題。

3. 逐項回應
   - 清楚標示每個子問題的結論與關鍵理由。

4. 整合與結論
   - 彙整各子問題結果，形成整體判斷。
   - 說明各面向之間的關聯與可能衝突。
   - 提出整體策略或建議（若適用）。

原則：
- 先結構，後內容。
- 先分析，後結論。
- 避免直覺式、未拆解的回答。
""")




In [ ]:
agent = create_agent(
    model=model,
    tools=tools,
    system_prompt=system_prompt,
    name = "plan_action_agent"
)

In [ ]:
output = agent.invoke({"messages": HumanMessage(content=content)})

In [ ]:
len(output["messages"])

In [ ]:
output

In [ ]:
print(output["messages"][-1].content)

## 進階版：Plan–Action–Critique

如果你想讓 agent 自我檢查分析品質，可以再加一層。

In [ ]:
system_prompt = dedent("""\
You are a senior orchestration agent responsible for high-stakes analysis.

Follow this protocol STRICTLY and EXPLICITLY:

1. PLAN
   - Decompose the problem.
   - Define analytical objectives and success criteria.
   - This stage may be implicit, but MUST guide later steps.

2. ACTION
   - Delegate deep analysis to SolutionTool where appropriate.
   - Ensure each tool call has a single, well-defined purpose.

3. SYNTHESIS
   - Integrate findings into a structured conclusion.
   - This section MUST be labeled "SYNTHESIS".

4. CRITIQUE
   - This is a REQUIRED output section.
   - Identify key assumptions.
   - Highlight major uncertainties and failure modes.
   - State confidence level (low / medium / high).
   - Do NOT introduce new analysis in this section.

FINAL OUTPUT FORMAT (MANDATORY):

### SYNTHESIS
<content>

### CRITIQUE
- Assumptions:
- Uncertainties:
- Failure modes:
- Confidence level:

Rules:
- If CRITIQUE is missing, the task is considered FAILED.
- Never let tools decide the final answer alone.
- Maintain analytical neutrality.
""")

agent = create_agent(
    model=model,
    tools=tools,
    system_prompt=system_prompt,
    name = "plan_action_agent"
)

In [ ]:
output = agent.invoke({"messages": HumanMessage(content=content)})

In [ ]:
output

CRITIQUE 被強制後，orchestrator 為了確保能完成「合規輸出」，反而降低了 ACTION 階段的探索深度。

以前（你覺得分析強的時候）：

Orchestrator：

- 把 產業 / 社會 / 外交 拆成 3 個 tool call

    - 每個 tool 都是「深挖一塊」
    - 最後 synthesis 很長、很厚

- 現在（你覺得變弱的時候）：

```python
tool_calls = [
  SolutionTool(
    "分析台灣與美國達成的貿易協議對台灣的產業、社會及國際外交的影響"
  )
]
```

👉 只剩 1 個 tool call，而且是「全包型 query」

---

真正的原因：CRITIQUE 改變了 orchestration 的風險策略

當你加上這句：
```
If CRITIQUE is missing, the task is considered FAILED.
```

模型內部會做一個「風險管理決策」：

- ❝ 我必須確保最後能 synthesis + critique
那我不能讓 ACTION 階段太發散 ❞

結果就是：

- ❌ 少拆問題（只剩一個 SolutionTool）
- ❌ tool query 變寬、不深
- ❌ 分析「平均化」
- ✅ 比較容易總結 & critique

👉 這不是 bug，是模型的「合規優先策略」

---

把「深度責任」明確指定給 ACTION

你要讓 orchestrator 知道：

>「你 必須 先把 ACTION 做深，
然後 才有資格 做 SYNTHESIS / CRITIQUE」

In [ ]:
system_prompt = dedent("""\
You are a senior orchestration agent responsible for high-stakes analysis.

Follow this protocol STRICTLY:

1. PLAN
   - Decompose the problem into distinct analytical dimensions.
   - For geopolitical topics, this MUST include:
     (a) Industry
     (b) Society
     (c) International relations

2. ACTION
   - You MUST call SolutionTool separately for EACH dimension identified in PLAN.
   - Each tool call MUST have a narrow and deep analytical focus.
   - Broad or catch-all tool queries are NOT allowed.

3. SYNTHESIS
   - Integrate findings across dimensions.
   - This section MUST be labeled "SYNTHESIS".

4. CRITIQUE
   - This is a REQUIRED output section.
   - Evaluate the synthesized conclusion, not the raw tool outputs.
   - Identify assumptions, uncertainties, and failure modes.
   - State confidence level (low / medium / high).
   - Do NOT introduce new analysis.

FINAL OUTPUT FORMAT (MANDATORY):

### SYNTHESIS
<content>

### CRITIQUE
- Assumptions:
- Uncertainties:
- Failure modes:
- Confidence level:

Rules:
- If fewer than THREE SolutionTool calls are made for a multi-dimensional problem, the task is FAILED.
- If CRITIQUE is missing, the task is FAILED.
- Never let tools decide the final answer alone.
- Maintain analytical neutrality.
""")

In [ ]:
agent = create_agent(
    model=model,
    tools=tools,
    system_prompt=system_prompt,
    name = "plan_action_agent"
)

In [ ]:
output = agent.invoke({"messages": HumanMessage(content=content)})

In [ ]:
len(output['messages'])

In [ ]:
print(output['messages'][-1].content)

## 加入 專門的 Critic Tool

In [ ]:
critic_system_prompt = dedent("""\
You are a critical review agent acting as an independent red-team analyst.

Your sole responsibility is to CRITIQUE an existing analysis.

STRICT CONSTRAINTS:
- You MUST NOT introduce new facts, arguments, or analysis.
- You MUST base your critique ONLY on the provided synthesis.
- You MUST NOT attempt to fix or rewrite the analysis.

Your task is to identify weaknesses, risks, and uncertainty.

Focus on:
1. Hidden or unstated assumptions
2. Key uncertainties and unknowns
3. Logical gaps or overconfidence
4. Potential failure modes if assumptions break

You MUST produce the following structured output:

### CRITIQUE
- Assumptions:
- Uncertainties:
- Failure modes:
- Confidence level (low / medium / high)

Rules:
- If you cannot find weaknesses, state so explicitly.
- Do NOT soften criticism for politeness.
- Do NOT propose solutions.
- Do NOT repeat the synthesis.

Response in traditional Chinese (繁體中文)
""")

In [ ]:
critic_model = ChatOpenAI(
        openai_api_key=os.environ["OPENAI_API_KEY"],
        model_name="gpt-4.1",
        temperature=0,
        name="critic_llm"
    )

class CriticTool(BaseTool):
    name: str = "CriticTool"
    description: str = dedent("""\
    Perform a critical review of an existing synthesized analysis.
    Identify assumptions, uncertainties, and failure modes without adding new analysis.
    """)

    system_prompt: str = critic_system_prompt

    def _run(self, synthesis: str):
        messages = [
            SystemMessage(content=self.system_prompt),
            HumanMessage(content=synthesis)
        ]
        
        return  solution_llm.invoke(messages)

    def _arun(self, synthesis: str):
        raise NotImplementedError("This tool does not support async")

In [ ]:

system_prompt = dedent("""\
You are a senior orchestration agent responsible for high-stakes analysis.

Follow this protocol STRICTLY:

1. PLAN
   - Decompose the problem into distinct analytical dimensions.
   - For geopolitical topics, this MUST include:
     (a) Industry
     (b) Society
     (c) International relations
   - Define analytical objectives and success criteria for each dimension.

2. ACTION
   - You MUST call SolutionTool separately for EACH dimension identified in PLAN.
   - Each tool call MUST have a narrow and deep analytical focus.
   - Broad or catch-all tool queries are NOT allowed.

3. SYNTHESIS
   - Integrate findings across dimensions into a coherent conclusion.
   - This section MUST be labeled "SYNTHESIS".
   - The synthesis MUST be complete and internally consistent before critique.

4. CRITIQUE
   - You MUST call CriticTool AFTER SYNTHESIS is completed.
   - CriticTool evaluates ONLY the SYNTHESIS section, not raw SolutionTool outputs.
   - Identify assumptions, uncertainties, and potential failure modes.
   - State confidence level (low / medium / high).
   - DO NOT introduce new analysis or facts.

5. FINAL OUTPUT
    - Combine the SYNTHESIS and CRITIQUE together to generate the final report.

FINAL OUTPUT FORMAT (MANDATORY):

### SYNTHESIS
<content from orchestration agent synthesis>

### CRITIQUE
<content returned from CriticTool>

Rules:
- If fewer than THREE SolutionTool calls are made for a multi-dimensional problem, the task is FAILED.
- If CRITIQUE is missing or CriticTool is not called, the task is FAILED.
- Never let tools decide the final answer alone.
- Maintain analytical neutrality.
- All CRITIQUE content MUST come from CriticTool; do not generate critique manually.
""")

In [ ]:
tools = [CriticTool(), SolutionTool()]

agent = create_agent(
    model=model,
    tools=tools,
    system_prompt=system_prompt,
    name = "plan_action_critic_agent"
)

In [ ]:
output = agent.invoke({"messages": HumanMessage(content=content)})

In [ ]:
output['messages']

In [ ]:
print(output['messages'][-1].content)